## Development Notes

Board: Rev A2.3

## SD Card 🛠️

```c
#define SD_CS     (15)
#define SD_MISO   (12)  // Tested on pin 34 due to GPIO 12 issue
#define SD_MOSI   (13)
#define SD_CLK    (14)
```

- Proof Of Concept ✅
- Need to burn eFuse due to `GPIO 12` being used for `VDD_SDIO` voltage selection, and the `HSPI_MISO` pin on this is not OK.

[eFuse Reference (mostly useless)][1] Steps to follow are here below

```sh
pip install cryptography ecdsa pyaml bitstring reedsolo
pio pkg exec -- espefuse.py --port COM4 summary
```

```raw
Vdd fuses:
XPD_SDIO_REG (BLOCK0)                              read for XPD_SDIO_REG                              = False R/W (0b0)
XPD_SDIO_TIEH (BLOCK0)                             If XPD_SDIO_FORCE & XPD_SDIO_REG                   = 1.8V R/W (0b0)
XPD_SDIO_FORCE (BLOCK0)                            Ignore MTDI pin (GPIO12) for VDD_SDIO on reset     = False R/W (0b0)

Flash voltage (VDD_SDIO) determined by GPIO12 on reset (High for 1.8V, Low/NC for 3.3V)
```

Burn eFuse to free `GPIO12`:

```sh
pio pkg exec -- espefuse.py --port COM4 burn_efuse XPD_SDIO_FORCE  # Do not consider GPIO12
pio pkg exec -- espefuse.py --port COM4 burn_efuse XPD_SDIO_TIEH   # Set VDD_SDIO to 3V3
pio pkg exec -- espefuse.py --port COM4 burn_efuse XPD_SDIO_REG    # Use XPD_SDIO_TIEH to set VDD_SDIO
```

```raw
Vdd fuses:
XPD_SDIO_REG (BLOCK0)                              read for XPD_SDIO_REG                              = True R/W (0b1)
XPD_SDIO_TIEH (BLOCK0)                             If XPD_SDIO_FORCE & XPD_SDIO_REG                   = 3.3V R/W (0b1)
XPD_SDIO_FORCE (BLOCK0)                            Ignore MTDI pin (GPIO12) for VDD_SDIO on reset     = True R/W (0b1)
```

- Moved to separate class (not defined yet) 🛠️

[1]: https://docs.espressif.com/projects/esp-idf/en/latest/esp32/api-reference/system/efuse.html

## Calendar LEDs 🛠️

- 🟡 <span style="color: red">**GPIO35 is only input**</span> Once again, GPIO check failed. Will not use this pin ⇒ oe will not be connected.
- Proof of concept using Adafruit library ✅
- Move to use VSPI